# Stock data analysis

In [ ]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI

In [ ]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


In [ ]:
openai = OpenAI()

In [ ]:
from urllib.parse import urlencode
import datetime

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [ ]:
class YahooFinanceWebsite:
    def __init__(self, stock_symbol):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.stock_symbol = stock_symbol.upper()

    def __build_url(self, params):
        base_url = f"https://finance.yahoo.com/quote/{self.stock_symbol}/history/"
        query_string = urlencode(params)
        return f"{base_url}?{query_string}"

    def get_stock_data(self):
        datetime_now = datetime.datetime.now()
        datetime_year_ago = datetime_now - datetime.timedelta(days=365)
        params = {"frequency": "1wk", "period1": datetime_year_ago.timestamp(), "period2": datetime_now.timestamp()}
        url = self.__build_url(params)
        response = requests.get(url, headers=headers)

        soup = BeautifulSoup(response.content, 'html.parser')
        
        title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        html_table_data = soup.find("table")

        return title, html_table_data

In [ ]:
def build_stock_analysis_prompt(stock_symbol, title, stock_table_data):
    sys_prompt = r"""You are an assistant that analyzes the contents of HTML formated table that contains data on a specific stock.
    The HTML table contains the date, open price, close price, low and highs aggregated for every week over one year timeframe.
    Ignoring text, tags or html attributes that might be navigation related. 
    Respond in Markdown format"""
    
    user_prompt = f"The data provided below in the HTML table format for {stock_symbol} from the Yahoo Finances.\
    Make the explaination easy enough for a newbie to understand. \
    Analyze and Summarize the trends on this stock:\n{stock_table_data}\n\n\
    Also, calculate the total returns in percentage one could have expected over this period."
    
    return [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt}
    ]

In [ ]:
def analyze_stock_trends(stock_symbol):
    stock_data_page = YahooFinanceWebsite(stock_symbol)
    title, stock_table_data = stock_data_page.get_stock_data()
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = build_stock_analysis_prompt(stock_symbol, title, stock_table_data)
    )
    return response.choices[0].message.content

def display_analysis(stock_symbol):
    display(Markdown(analyze_stock_trends(stock_symbol)))

In [ ]:
display_analysis("GOOG")

In [ ]:
display_analysis("PFE")

In [ ]:
display_analysis("AAPL")